In [1]:
from IPython.display import display, clear_output
import utils
import widgets
import ipywidgets as ipw
import shutil
import json

In [2]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

slabs_options = [object_key for object_key, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab"]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

material_selector = widgets.MaterialSelectionWidget()

sample_out_name_textbox = utils.Text(
    description = "Name", disabled = False, layout = ipw.Layout(width = '400px'), 
    placeholder = f"Write sample name here...", style = {'description_width': "110px"}
)
increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
# Function to create sample object inside openBIS using information selected in the app
def create_sample_action(b):
    samples_names = [sample.props["$name"] for sample in OPENBIS_SESSION.get_objects(type = "SAMPLE")]
    if sample_out_name_textbox.value in samples_names:
        display(utils.Javascript(data = f"alert('{'Sample name already exists!'}')"))
        return
    else:
        sample_parents = [] if material_selector.dropdown.value == -1 else [material_selector.dropdown.value]
        sample_props = {"$name": sample_out_name_textbox.value, "exists": True}
        utils.create_openbis_object(OPENBIS_SESSION, type="SAMPLE", collection="/MATERIALS/SAMPLES/SAMPLE_COLLECTION", props=sample_props, parents=sample_parents)
        display(utils.Javascript(data = "alert('Upload successful!')"))

def select_material_radio_change(change):
    material_type = material_selection_radio_button.value
    if material_type == "No material":
        with material_selector:
            clear_output()
            material_selector.dropdown.value = -1
            return
    
    material_selector.details_textbox.value = ''
    material_types = {}
    for object_key, object_info in CONFIG["objects"].items():
        if object_info["object_type"] == "slab":
            material_types[object_key] = (object_info["openbis_object_type"], object_info["placeholder"])
            
    with material_selector:
        clear_output()
        # Reset sample out name
        sample_out_name_textbox.value = ''
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_class, placeholder = material_types.get(material_type)
        material_selector.load_dropdown_box(material_class, placeholder)
        display(ipw.VBox(display_list))

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    if material_selector.dropdown_boxes.children[0].value == -1:
        material_selector.details_textbox.value = ''
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])
        return
    
    # Get material object information and dataset
    material_object = OPENBIS_SESSION.get_object(material_selector.dropdown_boxes.children[0].value)
    material_dataset = material_object.get_datasets()[0]
    
    # Get the object image preview
    if material_dataset:
        material_dataset.download(destination="images")
        material_selector.image_box.value = utils.read_file(f"images/{material_dataset.permId}/{material_dataset.file_list[0]}")
        # Erase file after downloading it
        shutil.rmtree(f"images/{material_dataset.permId}")
    else:
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])

    # Make a string with the property values of the object
    material_metadata_string = utils.get_metadata_string(OPENBIS_SESSION, material_object, "", CONFIG)

    material_selector.details_textbox.value = material_metadata_string
    sample_out_name_textbox.value = utils.convert_datetime_to_string(utils.get_current_datetime()) + f"_{material_object.props['$name']}"


# Create a sample object in openBIS

## Select material

In [ ]:
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown_boxes.children[0].observe(load_material_metadata, names = 'value')
display(material_selection_radio_button)
display(material_selector)


RadioButtons(layout=Layout(width='300px'), options=('No material', 'Crystal', 'Wafer substrate', '2D-layer mat…

## Sample out

In [ ]:
create_button.on_click(create_sample_action)
quit_button.on_click(close_notebook)
display(sample_out_name_textbox, save_close_buttons_hbox)
display(increase_buttons_size)